In [ ]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
df = pd.read_csv('smart_home_data.csv')
df.head()

,Timestamp,TimeOfDay,Bedroom1_Light,Bedroom1_AC,Bedroom1_Computer,Bedroom2_Light,Bedroom2_AC,Bedroom2_Computer,Bedroom3_Light,Bedroom3_AC,...,Bath2_WashingMachine,Bath2_Heater,Bath2_Light,Bath2_ExhaustFan,Living_Light,Living_Fan,Living_TV,Living_HomeTheater,Outdoor_Light,Total_Usage_W
0,2024-01-01 00:00:00,Night,0.0,0.00,0.0,0.0,1495.63,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,15.41,1633.45
1,2024-01-01 00:05:00,Night,0.0,1502.68,0.0,0.0,1501.41,195.15,0.0,1502.97,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,14.38,4716.59
2,2024-01-01 00:10:00,Night,0.0,1495.59,0.0,0.0,1499.06,0.00,0.0,1497.63,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,11.96,5307.53
3,2024-01-01 00:15:00,Night,0.0,1495.67,0.0,0.0,1500.33,0.00,0.0,1501.70,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,14.66,4512.36
4,2024-01-01 00:20:00,Night,0.0,1502.10,0.0,0.0,0.00,0.00,0.0,1502.08,...,0.0,0.0,0.0,0.0,0.0,72.07,0.0,0.0,17.52,3093.77


In [ ]:
# Preprocess Data
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Hour'] = df['Timestamp'].dt.hour
df['Minute'] = df['Timestamp'].dt.minute
df['DayOfWeek'] = df['Timestamp'].dt.dayofweek

In [ ]:
# Features (Inputs) and Target (Output)
X = df[['Hour', 'Minute', 'DayOfWeek']]
y = df['Total_Usage_W']

In [ ]:
# Train Model
print("Training Random Forest Model...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

Training Random Forest Model...


RandomForestRegressor(random_state=42)

In [ ]:
# Validate
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f"Model Trained. Mean Absolute Error: {mae:.2f} Watts")

Model Trained. Mean Absolute Error: 1475.51 Watts


In [1]:
import pandas as pd
import numpy as np
import random
import joblib
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

def get_appliances_config():
    return {
        'Living Room Light': 15, 'Living Room TV': 120, 'Living Room AC': 1200,
        'Kitchen Oven': 2500, 'Kitchen Fridge': 700, 'Kitchen Light': 30,
        'Bedroom 1 Light': 30, 'Bedroom 1 AC': 800, 'Bedroom 1 PC': 560,
        'Bedroom 2 Light': 15, 'Bedroom 2 AC': 800, 'Bedroom 2 TV': 80,
        'Bathroom 1 Light': 15, 'Bathroom 1 Exhaust Fan': 40,
        'Bathroom 2 Light': 15, 'Bathroom 2 Exhaust Fan': 40, 'Washing Machine':1200,
        'Bathroom 1 Hot Water Shower': 8000, 'Bathroom 2 Hot Water Shower': 8000,
        'Garage Light': 15, 'Outdoor Lights': 80
    }

def generate_structured_data(filename='smart_home_data.csv'):
    print("Generating Structured (Learnable) Data...")
    appliances = get_appliances_config()
    start = datetime(2024, 1, 1)
    end = start + timedelta(days=30)
    current = start
    data = []

    while current < end:
        h = current.hour
        m = current.minute
        wd = current.weekday() # 0=Mon, 6=Sun

        # Initialize all devices to OFF
        state = {k: 0 for k in appliances}

        # --- DAILY ROUTINES ---

        # SLEEP (23:00 - 06:00)
        if h >= 23 or h < 6:
            state['AC'] = 1        # AC is always ON at night
            state['Fridge'] = 1    # Fridge runs
            state['Light'] = 0     # Lights OFF

        # MORNING (06:00 - 09:00)
        elif 6 <= h < 9:
            state['AC'] = 0
            state['Light'] = 1     # Lights ON
            state['Fridge'] = 1
            if 7 <= h < 8:         # Breakfast time
                state['Oven'] = 1
                state['Toaster'] = 1

        # WORK DAY (09:00 - 17:00)
        elif 9 <= h < 17:
            if wd < 5: # Weekday: House Empty
                state['AC'] = 0
                state['Light'] = 0
                state['Fridge'] = 1
            else: # Weekend: Home
                state['AC'] = 1
                state['TV'] = 1
                state['Computer'] = 1

        # EVENING (17:00 - 23:00)
        elif 17 <= h < 23:
            state['Light'] = 1
            if 18 <= h < 19: state['Oven'] = 1      # Dinner
            if 19 <= h < 22: state['TV'] = 1        # TV Time
            if 20 <= h < 23: state['AC'] = 1        # Cooling for sleep
            state['Fridge'] = 1

        # Calculate Total Watts
        total_watts = 0
        for app, is_on in state.items():
            if is_on and app in appliances:
                # Add small noise
                power = appliances[app] + random.uniform(-5, 5)
                total_watts += power

        # Add Extra Random Load (Router, Chargers, etc.)
        total_watts += random.uniform(20, 40)

        data.append({
            'Timestamp': current,
            'Hour': h,
            'Minute': m,
            'DayOfWeek': wd,
            'Total_Usage_W': round(total_watts, 2)
        })
        current += timedelta(minutes=5)

    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Saved {filename} ({len(df)} rows)")
    return df

def train_model(df):
    print("2. Training AI Model...")
    X = df[['Hour', 'Minute', 'DayOfWeek']]
    y = df['Total_Usage_W']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Check Accuracy
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    print(f"Model MAE: {mae:.2f} Watts")

    if mae < 100:
        print("The model has learned the daily patterns")
    else:
        print(f"Error is high MAE: {mae:.2f} Watts")

    joblib.dump(model, 'energy_model.pkl')
    print("Model saved as 'energy_model.pkl'")

if __name__ == "__main__":
    df = generate_structured_data()
    train_model(df)

Generating Structured (Learnable) Data...
Saved smart_home_data.csv (8640 rows)
2. Training AI Model...
Model MAE: 5.59 Watts
The model has learned the daily patterns
Model saved as 'energy_model.pkl'


In [ ]:
import pandas as pd
import numpy as np
import random
import joblib
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

def generate_family_routine_data(filename='smart_home_data.csv'):
    print("1. Generating Data based on Family Routine...")

    # EXACT Wattage from your scenario
    appliances = {
        'Living Room Light': 15, 'Living Room TV': 120, 'Living Room AC': 1200,
        'Kitchen Oven': 2500, 'Kitchen Fridge': 700, 'Kitchen Light': 30,
        'Bedroom 1 Light': 30, 'Bedroom 1 AC': 800, 'Bedroom 1 PC': 560,
        'Bedroom 2 Light': 15, 'Bedroom 2 AC': 800, 'Bedroom 2 TV': 80,
        'Bathroom 1 Light': 15, 'Bathroom 1 Exhaust Fan': 40, 'Bathroom 1 Heater': 8000,
        'Bathroom 2 Light': 15, 'Bathroom 2 Exhaust Fan': 40, 'Bathroom 2 Heater': 8000,
        'Washing Machine': 1200, 'Garage Light': 15, 'Outdoor Lights': 80
    }

    start = datetime(2024, 1, 1)
    end = start + timedelta(days=30)
    current = start
    data = []

    while current < end:
        h = current.hour
        m = current.minute
        wd = current.weekday() # 0=Mon, ... 5=Sat, 6=Sun
        is_weekend = (wd >= 5)

        # Initialize state (Everything OFF)
        state = {k: 0 for k in appliances}

        # --- BASE LOADS ---
        # Outdoor lights: 6pm to 6am
        if h >= 18 or h < 6:
            state['Outdoor Lights'] = 1

        # Fridge cycles (Approx 50% duty cycle)
        if random.random() < 0.5:
            state['Kitchen Fridge'] = 1

        # --- WEEKDAY ROUTINE ---
        if not is_weekend:
            # 1. MORNING RUSH (05:00 - 06:30)
            if 5 <= h < 7:
                # Dad wakes up 05:00
                if h == 5 and m < 30:
                    state['Bedroom 1 Light'] = 1
                    state['Kitchen Light'] = 1
                    state['Kitchen Oven'] = 1 # Breakfast/Lunch prep
                    # Dad Bath
                    if 10 <= m < 25:
                        state['Bathroom 1 Light'] = 1
                        state['Bathroom 1 Heater'] = 1
                        state['Bathroom 1 Exhaust Fan'] = 1

                # Kids & Mom (05:30 - 06:30)
                if (h == 5 and m >= 30) or (h == 6 and m < 30):
                    state['Bedroom 2 Light'] = 1 # Kids up
                    state['Kitchen Light'] = 1
                    state['Kitchen Oven'] = 1

                    # Kids Bath (Hot Water)
                    state['Bathroom 2 Light'] = 1
                    state['Bathroom 2 Heater'] = 1
                    state['Bathroom 2 Exhaust Fan'] = 1

                    # Mom Bath (Hot Water)
                    if h == 6 and m < 20:
                        state['Bathroom 1 Light'] = 1
                        state['Bathroom 1 Heater'] = 1

            # 2. HOUSE EMPTY / WORKER (09:00 - 14:00)
            elif 9 <= h < 14:
                # Worker Schedule
                if 9 <= h < 11:
                    state['Washing Machine'] = 1 # Washing clothes

                # Cleaning & Cooking
                if random.random() < 0.3: state['Kitchen Light'] = 1
                if 11 <= h < 13: state['Kitchen Oven'] = 1 # Dinner prep
                if random.random() < 0.2: state['Living Room Light'] = 1 # Cleaning

            # 3. AFTERNOON: KIDS HOME (14:00 - 19:00)
            elif 14 <= h < 19:
                # Bathing (NO Hot Water)
                if 14 <= h < 15:
                    state['Bathroom 1 Light'] = 1
                    state['Bathroom 2 Light'] = 1

                # Entertainment / Noon Heat
                state['Bedroom 2 AC'] = 1 # It's noon/hot
                state['Bedroom 2 TV'] = 1
                state['Bedroom 1 PC'] = 1 # gaming/homework

            # 4. EVENING: PARENTS HOME (19:00 - 22:00)
            elif 19 <= h < 22:
                state['Living Room Light'] = 1
                state['Kitchen Light'] = 1

                # Dinner heating
                if 19 <= h < 20:
                    state['Kitchen Oven'] = 1
                    # Parents Bath
                    if m < 30: state['Bathroom 1 Light'] = 1

                # Relaxation
                if 20 <= h < 22:
                    state['Living Room TV'] = 1
                    state['Living Room AC'] = 1
                    state['Bedroom 1 PC'] = 1

            # 5. NIGHT (22:00 - 05:00)
            if h >= 22 or h < 5:
                state['Bedroom 1 AC'] = 1
                state['Bedroom 2 AC'] = 1

        # --- WEEKEND ROUTINE (Sat-Sun) ---
        else:
            # Wake up later (08:00 - 09:00)
            if 8 <= h < 10:
                state['Kitchen Light'] = 1
                state['Kitchen Oven'] = 1 # Breakfast

            # Worker (10:00 onwards)
            if 10 <= h < 13:
                state['Washing Machine'] = 1

            # Day (Relaxed usage)
            if 10 <= h < 22:
                # Random AC/TV usage throughout day
                if random.random() < 0.6: state['Living Room AC'] = 1
                if random.random() < 0.5: state['Living Room TV'] = 1
                if random.random() < 0.4: state['Bedroom 1 AC'] = 1
                if random.random() < 0.4: state['Bedroom 1 PC'] = 1

            # Night
            if h >= 22 or h < 8:
                state['Bedroom 1 AC'] = 1
                state['Bedroom 2 AC'] = 1

        # --- CALCULATE TOTAL WATTS ---
        total_watts = 0
        for name, is_on in state.items():
            if is_on:
                total_watts += appliances[name]

        # Add random noise (fluctuation)
        total_watts += random.uniform(-10, 10)

        data.append({
            'Timestamp': current,
            'Hour': h,
            'Minute': m,
            'DayOfWeek': wd,
            'Total_Usage_W': max(0, round(total_watts, 2))
        })
        current += timedelta(minutes=5)

    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Saved {filename} ({len(df)} rows)")
    return df

def train_model(df):
    print("2. Training AI Model...")
    X = df[['Hour', 'Minute', 'DayOfWeek']]
    y = df['Total_Usage_W']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)

    # Check Accuracy
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    print(f"Model MAE: {mae:.2f} Watts")

    if mae < 100:
        print("The model has learned the daily patterns")
    else:
        print(f"Error is high MAE: {mae:.2f} Watts")

    joblib.dump(model, 'energy_model.pkl')
    print("Model saved as 'energy_model.pkl'")

if __name__ == "__main__":
    df = generate_family_routine_data()
    train_model(df)

1. Generating Data based on Family Routine...
Saved smart_home_data.csv (8640 rows)
2. Training AI Model...
Model MAE: 418.41 Watts
Error is high MAE: 418.41 Watts
Model saved as 'energy_model.pkl'


In [2]:
import pandas as pd
import numpy as np
import random
import joblib
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

def generate_family_routine_data(filename='smart_home_data.csv'):
    print("1. Generating Data based on Family Routine...")

    appliances = {
        'Living Room Light': 15, 'Living Room TV': 120, 'Living Room AC': 1200,
        'Kitchen Oven': 2500, 'Kitchen Fridge': 700, 'Kitchen Light': 30,
        'Bedroom 1 Light': 30, 'Bedroom 1 AC': 800, 'Bedroom 1 PC': 560,
        'Bedroom 2 Light': 15, 'Bedroom 2 AC': 800, 'Bedroom 2 TV': 80,
        'Bathroom 1 Light': 15, 'Bathroom 1 Exhaust Fan': 40, 'Bathroom 1 Heater': 8000,
        'Bathroom 2 Light': 15, 'Bathroom 2 Exhaust Fan': 40, 'Bathroom 2 Heater': 8000,
        'Washing Machine': 1200, 'Garage Light': 15, 'Outdoor Lights': 80
    }

    start = datetime(2024, 1, 1)
    end = start + timedelta(days=30)
    current = start
    data = []

    while current < end:
        h = current.hour
        m = current.minute
        wd = current.weekday()
        is_weekend = (wd >= 5)

        state = {k: 0 for k in appliances}

        # --- BASE LOADS ---
        if h >= 18 or h < 6:
            state['Outdoor Lights'] = 1

        # FIX 1: Fridge Cycle
        # Instead of random, it runs for the first 30 mins of every hour
        if m < 30:
            state['Kitchen Fridge'] = 1

        # --- WEEKDAY ROUTINE ---
        if not is_weekend:
            # 1. MORNING RUSH (05:00 - 06:30)
            if 5 <= h < 7:
                if h == 5 and m < 30: # Dad
                    state['Bedroom 1 Light'] = 1
                    state['Kitchen Light'] = 1
                    state['Kitchen Oven'] = 1
                    if 10 <= m < 25:
                        state['Bathroom 1 Light'] = 1
                        state['Bathroom 1 Heater'] = 1
                        state['Bathroom 1 Exhaust Fan'] = 1

                if (h == 5 and m >= 30) or (h == 6 and m < 30): # Mom & Kids
                    state['Bedroom 2 Light'] = 1
                    state['Kitchen Light'] = 1
                    state['Kitchen Oven'] = 1
                    state['Bathroom 2 Light'] = 1
                    state['Bathroom 2 Heater'] = 1
                    state['Bathroom 2 Exhaust Fan'] = 1
                    if h == 6 and m < 20:
                        state['Bathroom 1 Light'] = 1
                        state['Bathroom 1 Heater'] = 1

            # 2. HOUSE EMPTY / WORKER (09:00 - 14:00)
            elif 9 <= h < 14:
                if 9 <= h < 11: state['Washing Machine'] = 1

                # FIX 2: Cleaning Schedule
                # Cleaning happens on Mon, Wed, Fri only
                if wd in [0, 2, 4]:
                    state['Kitchen Light'] = 1
                    if 10 <= h < 11: state['Living Room Light'] = 1

                if 11 <= h < 13: state['Kitchen Oven'] = 1

            # 3. AFTERNOON (14:00 - 19:00)
            elif 14 <= h < 19:
                if 14 <= h < 15:
                    state['Bathroom 1 Light'] = 1
                    state['Bathroom 2 Light'] = 1

                state['Bedroom 2 AC'] = 1
                state['Bedroom 2 TV'] = 1
                state['Bedroom 1 PC'] = 1

            # 4. EVENING (19:00 - 22:00)
            elif 19 <= h < 22:
                state['Living Room Light'] = 1
                state['Kitchen Light'] = 1
                if 19 <= h < 20:
                    state['Kitchen Oven'] = 1
                    if m < 30: state['Bathroom 1 Light'] = 1

                if 20 <= h < 22:
                    state['Living Room TV'] = 1
                    state['Living Room AC'] = 1
                    state['Bedroom 1 PC'] = 1

            # 5. NIGHT (22:00 - 05:00)
            if h >= 22 or h < 5:
                state['Bedroom 1 AC'] = 1
                state['Bedroom 2 AC'] = 1

        # --- WEEKEND ROUTINE ---
        else:
            if 8 <= h < 10:
                state['Kitchen Light'] = 1
                state['Kitchen Oven'] = 1

            if 10 <= h < 13:
                state['Washing Machine'] = 1

            # FIX 3: Predictable Weekend Relaxation
            if 10 <= h < 22:
                # Even hours = AC, Odd hours = TV
                if h % 2 == 0:
                    state['Living Room AC'] = 1
                    state['Bedroom 1 AC'] = 1
                else:
                    state['Living Room TV'] = 1
                    state['Bedroom 1 PC'] = 1

            if h >= 22 or h < 8:
                state['Bedroom 1 AC'] = 1
                state['Bedroom 2 AC'] = 1

        # Calc Total
        total_watts = 0
        for name, is_on in state.items():
            if is_on: total_watts += appliances[name]

        # Small noise for voltage fluctuation
        total_watts += random.uniform(-5, 5)

        data.append({
            'Timestamp': current,
            'Hour': h, 'Minute': m, 'DayOfWeek': wd,
            'Total_Usage_W': max(0, round(total_watts, 2))
        })
        current += timedelta(minutes=5)

    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Saved {filename} ({len(df)} rows)")
    return df

def train_model(df):
    print("2. Training AI Model...")
    X = df[['Hour', 'Minute', 'DayOfWeek']]
    y = df['Total_Usage_W']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    print(f"Model MAE: {mae:.2f} Watts")

    if mae < 100:
        print("The model has learned the daily patterns")
    else:
        print(f"MAE is {mae:.2f}W")

    joblib.dump(model, 'energy_model.pkl')
    print("Model saved as 'energy_model.pkl'")

if __name__ == "__main__":
    df = generate_family_routine_data()
    train_model(df)

1. Generating Data based on Family Routine...
Saved smart_home_data.csv (8640 rows)
2. Training AI Model...
Model MAE: 2.76 Watts
The model has learned the daily patterns
Model saved as 'energy_model.pkl'
